In [ ]:
from IPython.display import clear_output
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import time


In [34]:


AI_SYMBOL = 1  # 'X'
PLAYER_SYMBOL = -1  # 'O'

# Function to create the model
def create_model():
    model = Sequential([
        Dense(64, input_dim=9, activation='relu'),
        Dense(64, activation='relu'),
        Dense(9, activation='linear')
    ])
    model.compile(optimizer=Adam(), loss='mse')
    return model

# Function to check for a win
def check_win(board, symbol):
    win_conditions = [(0, 1, 2), (3, 4, 5), (6, 7, 8),  # Rows
                      (0, 3, 6), (1, 4, 7), (2, 5, 8),  # Columns
                      (0, 4, 8), (2, 4, 6)]  # Diagonals
    return any(board[a] == board[b] == board[c] == symbol for a, b, c in win_conditions)

# Function to check for a draw
def check_draw(board):
    return all(cell != 0 for cell in board)

# Function for the AI to make a move
def ai_move(board, model):
    empty_cells = [i for i, cell in enumerate(board) if cell == 0]
    if empty_cells:
        predictions = model.predict(np.array([board]))[0]
        best_move = max(empty_cells, key=lambda x: predictions[x])
        board[best_move] = AI_SYMBOL
    # Clear the output and show the updated board
    clear_output(wait=False)
    print_board(board)

# Function for the player to make a move
def player_move(board):
    while True:
        try:
            move = int(input("Enter your move (1-9):   if it not show or bug just pass ENTER")) - 1
            if 0 <= move < 9 and board[move] == 0:
                board[move] = PLAYER_SYMBOL
                break
            else:
                print("Invalid move. Try again.")
        except ValueError:
            print("Please enter a number between 1 and 9.")
            print("if it not show board just pass ENTER")
    # Clear the output and show the updated board
    clear_output(wait=True)
    print_board(board)

# Function to print the board
def print_board(board):
    symbols = {1: 'X', -1: 'O', 0: ' '}
    for i in range(9):
        if i % 3 == 0 and i > 0:
            print("\n--+---+--")
        print(f" {symbols[board[i]]} ", end='')
        if i % 3 < 2:
            print("|", end='')
    print()

# Training the model with synthetic data
def train_model(model, batch_size=32, num_games=10000):
    X = []
    y = []
    print("Training the AI model... Please wait.")


    for game in range(num_games):
        board = np.random.choice([0, AI_SYMBOL, PLAYER_SYMBOL], size=9)
        if not check_win(board, AI_SYMBOL) and not check_win(board, PLAYER_SYMBOL):
            empty_cells = [i for i, cell in enumerate(board) if cell == 0]
            if empty_cells:
                move = np.random.choice(empty_cells)
                board_copy = board.copy()
                board_copy[move] = AI_SYMBOL
                reward = 0
                

                while True:
                    
                    if check_win(board_copy, PLAYER_SYMBOL):
                        reward = -1
                        break
                    elif check_draw(board_copy):
                        reward = 0
                        break

                    ai_move(board_copy, model)
                    if check_win(board_copy, AI_SYMBOL):
                        reward = 1
                        break
                    elif check_draw(board_copy):
                        reward = 0
                        break
                print("Training the AI model... Please wait.")
                X.append(board)
                y.append(reward * np.array(board_copy))
                clear_output(wait=True)

    if X:
        X = np.array(X)
        y = np.array(y)
        model.fit(X, y, epochs=20, batch_size=batch_size, verbose=1)
        print("wait for ai train")
        model.save("tic_tac_toe_model.h5")
    else:
        print("No training data generated. Adjust game simulation or data generation.")

# Main function to play the game
def main():
    model = create_model()
   
    train_model(model, batch_size=64, num_games=1000)  # Train with a limited number of games for simplicity

    player_score = 0
    ai_score = 0

    while True:
        board = [0] * 9
        clear_output(wait=True)  # Clear previous output (useful in Jupyter Notebook)
        print("Welcome to Tic-Tac-Toe!")
        print("Initial board state:")
        print_board(board)

        while True:
            # AI move
            ai_move(board, model)
            if check_win(board, AI_SYMBOL):
                print("AI (X) wins!")
                ai_score += 1
                break
            if check_draw(board):
                print("It's a draw!")
                break

            # Player move
            player_move(board)
            if check_win(board, PLAYER_SYMBOL):
                print("Player (O) wins!")
                player_score += 1
                break
            if check_draw(board):
                print("It's a draw!")
                break

        # Print final board state
        print("Final board state:")
        print_board(board)

        # Print scores
        print(f"Scores:\nAI (X): {ai_score}\nPlayer (O): {player_score}")

        play_again = input("Do you want to play again? (yes/no): ")
        if play_again == 'yes':
            main()
        else :
            break

if __name__ == "__main__":
    main()


 O | O | O 
--+---+--
   | X | X 
--+---+--
   | X |   
Player (O) wins!
Final board state:
 O | O | O 
--+---+--
   | X | X 
--+---+--
   | X |   
Scores:
AI (X): 0
Player (O): 1
